# Qwen Fine-Tuning on Google Colab
Train on Colab's GPU, download weights to your laptop

In [ ]:
# Install dependencies
!pip install -q transformers datasets peft bitsandbytes accelerate trl

In [ ]:
# Import libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
# Configuration
MODEL_NAME = "Qwen/Qwen2.5-Coder-7B-Instruct"
OUTPUT_DIR = "./qwen-finetuned"

# LoRA config - keeps training lightweight
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
# Load model and tokenizer
print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Prepare for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

print(f"Trainable parameters: {model.print_trainable_parameters()}")

In [ ]:
# Load training data
# Option 1: Use existing dataset
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train[:1000]")

# Option 2: Upload your own data (uncomment below)
# from google.colab import files
# uploaded = files.upload()
# dataset = load_dataset('json', data_files='your_data.json')

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_steps=100,
    logging_steps=10,
    save_total_limit=2,
    warmup_steps=50,
)

In [ ]:
# Create trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    peft_config=lora_config,
    dataset_text_field="output",  # Adjust based on your dataset
    max_seq_length=512,
    tokenizer=tokenizer,
)

# Start training
print("Starting training...")
trainer.train()

In [ ]:
# Save LoRA adapters (only ~100MB!)
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"Model saved to {OUTPUT_DIR}")
print("Download this fversion 1.0.1er to your laptop!")

In [ ]:
# Test the fine-tuned model
test_prompt = "Write a Python function to calculate fibonacci numbers"

inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=200)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Test output:")
print(result)

In [ ]:
# Download to your laptop
from google.colab import files
import shutil

# Zip the output
shutil.make_archive('qwen-finetuned', 'zip', OUTPUT_DIR)
files.download('qwen-finetuned.zip')

print("Download complete! Extract and place in hybrid_llm/models/")